# Playground 5:  Segmentation workflow for LAMP-1

This notebook contains the workflow for LAMP-1, and serves as a starting point for developing a classic segmentation workflow if your data shows similar morphology as LAMP-1.

----------------------------------------

Cell Structure Observations:

* [LAMP-1](https://www.allencell.org/cell-observations/category/lamp1)

----------------------------------------

Key steps of the workflows:

* Auto-Contrast intensity normalization
* 2D Gaussian smoothing slice by slice
* 2D Filament filter 
* 2D Spot filter 
* Hole filling
* Size thresholding

In [ ]:
import numpy as np

# package for 3d visualization
from itkwidgets import view                              
from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 12]

# package for io 
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter

# function for core algorithm
from aicssegmentation.core.vessel import filament_2d_wrapper
from aicssegmentation.core.seg_dot import dot_2d_slice_by_slice_wrapper
from aicssegmentation.core.utils import hole_filling
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_slice_by_slice
from skimage.morphology import remove_small_objects, watershed, dilation, erosion, ball     

## Loading the data

In [ ]:
FILE_NAME = '../../data/LAMP1/3500001519_100X_20171110_1-Scene-07-P21-E06.czi'

reader = AICSImage(FILE_NAME) 
IMG = reader.data.astype(np.float32)

print(IMG.shape)

## Preview of the image

In [ ]:
N_CHANNELS = IMG.shape[2]
MID_SLICE = np.int(0.5*IMG.shape[3])

fig, ax = plt.subplots(1, N_CHANNELS, figsize=(18,16), dpi=72, facecolor='w', edgecolor='k')
if N_CHANNELS==1:
    ax.axis('off')
    ax.imshow(IMG[0,0,0,MID_SLICE,:,:], cmap=plt.cm.gray)
else:
    for channel in range(N_CHANNELS):
        ax[channel].axis('off')
        ax[channel].imshow(IMG[0,0,channel,MID_SLICE,:,:], cmap=plt.cm.gray)

In [ ]:
#####################
structure_channel = 3
#####################

struct_img0 = IMG[0,0,structure_channel,:,:,:].copy()
view(single_fluorescent_view(struct_img0))

## Image segmentation

### Step 1: Pre-Processing

About selected algorithms and tuned parameters

* **Intensity normalization**: Parameter `intensity_scaling_param` has two options: two values, say `[A, B]`, or single value, say `[K]`. For the first case, `A` and `B` are non-negative values indicating that the full intensity range of the stack will first be cut-off into **[mean - A * std, mean + B * std]** and then rescaled to **[0, 1]**. The smaller the values of `A` and `B` are, the higher the contrast will be. For the second case, `K`>0 indicates min-max Normalization with an absolute intensity upper bound `K` (i.e., anything above `K` will be chopped off and reset as the minimum intensity of the stack) and `K`=0 means min-max Normalization without any intensity bound.

    * Parameter for LAMP1:  `intensity_scaling_param = [9, 19]`


* **Smoothing**:  2D gaussian smoothing slice by slice with `gaussian_smoothing_sigma = 1`. The large the value is, the more the image will be smoothed. 

In [ ]:
################################
## PARAMETERS for this step ##
intensity_scaling_param = [3, 19]
gaussian_smoothing_sigma = 1
################################
# intensity normalization
struct_img = intensity_normalization(struct_img0, scaling_param=intensity_scaling_param)

# smoothing with 2d gaussian filter slice by slice 
structure_img_smooth = image_smoothing_gaussian_slice_by_slice(struct_img, sigma=gaussian_smoothing_sigma)

In [ ]:
view(single_fluorescent_view(structure_img_smooth))

#### If the contrast looks too off, you can tune the normalization parameters.

We have a function to give you some suggestions. If you have certain preference, you can adjust the values based on the suggestion.

***After you decide the parameters, you have to re-run the code above with the new parameter*** `intensity_scaling_param = ` 

In [ ]:
from aicssegmentation.core.pre_processing_utils import suggest_normalization_param
suggest_normalization_param(struct_img0)

### Step 2: Core Algorithm

#### step 2.1: Apply 2D spot filter slice by slice to detect spots

Parameter syntax: `[[scale_1, cutoff_1], [scale_2, cutoff_2], ....]` 
* `scale_x` is set based on the estimated radius of your target spotty shape. For example, if visually the diameter of the spotty objects is usually 3~4 pixels, then you may want to set `scale_x` as `1` or something near `1` (like `1.25`). Multiple scales can be used, if you have objects of very different sizes.  
* `cutoff_x` is a threshold applied on the actual filter reponse to get the binary result. Smaller `cutoff_x` may yielf fatter segmentation, while larger `cutoff_x` could be less permisive and yield less objects and slimmer segmentation. 

Parameter for LAMP1:  `s2_param =[[5,0.09], [2.5,0.07], [1,0.01]]`

In [ ]:
################################
## PARAMETERS for this step ##
s2_param = [[5,0.09], [2.5,0.07], [1,0.01]]
################################

bw_spot = dot_2d_slice_by_slice_wrapper(structure_img_smooth, s2_param)

In [ ]:
viewer_bw_spot = view(segmentation_quick_view(bw_spot))
viewer_bw_spot

##### After quickly visualizing the segmentation results, you can also visualize the segmentation and original image side by side
##### You may select an ROI to inspect the details

* Option 1: Easy ROI selection, but NOT recommended if you are using a laptop

You can select an ROI in above visualization ('viewer_bw_spot'); otherwise, the default ROI is the full image

[See this video for How to select ROI](https://www.youtube.com/watch?v=ZO8ey6-tF_0&index=3&list=PL2lHcsoU0YJsh6f8j2vbhg2eEpUnKEWcl)

* Option 2: Manually type in ROI coordinates

Type in the coordinates of upper left corner and lower right corner of the ROI in the form of [Upper_Left_X, Upper_Left_Y, Lower_right_X, Lower_right_Y]. 

In [ ]:
# Option 1:
view(seg_fluo_side_by_side(struct_img,bw_spot,roi=['ROI',viewer_bw_spot.roi_slice()]))

# Option 2: 
# view(seg_fluo_side_by_side(struct_img,bw_spot,roi=['M',[570,370,730,440]]))

##### Is the segmentation satisfactory? Here are some possible criteria:

--------------------
* Is there any spot should be detected but not? Try to reduce `cutoff_x`
* Is there any object should not be detected but actually appear in the result? Try to increase `cutoff_x` or try a larger `scale_x`
* Is the segmented size of the spots fatter than it should be? Try to increase `cutoff_x` or try a smaller `scale_x`
* Is there any spot that should be solid but segmented as a ring? Try to increase `scale_x`
* Are you observing spots with very different sizes? Try multiple sets of `scale_x` and `cutoff_x` 
------------------------

**If good, go to step 2.2**

#### step 2.2: Apply 2D filament filter slice by slice to filaments

Parameter syntax: `[[scale_1, cutoff_1], [scale_2, cutoff_2], ....]` 
* `scale_x` is set based on the estimated width of your target curvilinear shape. For example, if visually the width of the objects is usually 3~4 pixels, then you may want to set `scale_x` as `1` or something near `1` (like `1.25`). Multiple scales can be used, if you have objects of very different sizes.  
* `cutoff_x` is a threshold applied on the actual filter reponse to get the binary result. Smaller `cutoff_x` may yielf fatter segmentation, while larger `cutoff_x` could be less permisive and yield less objects and slimmer segmentation. 

Parameter for LAMP1:  `f2_param = [[1, 0.15]]`

In [ ]:
################################
## PARAMETERS for this step ##
f2_param = [[1, 0.15]]
################################

bw_filament = filament_2d_wrapper(structure_img_smooth, f2_param)

In [ ]:
view(segmentation_quick_view(bw_filament))

##### Is the segmentation satisfactory? Here are some possible criteria:

----------------
* Is there any object should be detected but not? Try to reduce `cutoff_x`
* Is there any object should not be detected but actually appear in the result? Try to increase `cutoff_x` or try a larger `scale_x`
* Is the segmented width of the objects is fatter than it should be? Try to increase `cutoff_x` or try a smaller `scale_x`
* Is there any object that should be solid but segmented as fragmented pieces? Try to increase `scale_x`
* Are you observing objects with very different width? Try multiple sets of `scale_x` and `cutoff_x` 
-------------

#### step 2.3: merge the results

In [ ]:
bw = np.logical_or(bw_spot, bw_filament)

#### Step 3: Post-Processing 

In [ ]:
################################
## PARAMETERS for this step ##
fill_2d = True
fill_max_size = 1600
minArea = 15
################################

bw_fill = hole_filling(bw, 0, fill_max_size, fill_2d)

seg = remove_small_objects(bw_fill>0, min_size=minArea, connectivity=1, in_place=False)

## Result inspection

In [ ]:
viewer_final = view(segmentation_quick_view(seg))
viewer_final

### You can also focus your inspection on a small ROI

* Option 1: Easy ROI selection, but NOT recommended if you are using a laptop

You can select an ROI in above visualization ('viewer_final'); otherwise, the default ROI is the full image

[See this video for How to select ROI](https://www.youtube.com/watch?v=ZO8ey6-tF_0&index=3&list=PL2lHcsoU0YJsh6f8j2vbhg2eEpUnKEWcl)

* Option 2: Manually type in ROI coordinates

Type in the coordinates of upper left corner and lower right corner of the ROI in the form of [Upper_Left_X, Upper_Left_Y, Lower_right_X, Lower_right_Y]. 

In [ ]:
# Option 1: 
view(seg_fluo_side_by_side(struct_img, seg, roi=['ROI',viewer_final.roi_slice()]))

# Option 2: 
#view(seg_fluo_side_by_side(struct_img, seg, roi=['M',[267,474, 468, 605]]))

### You may also physically save the segmentation results into a ome.tif file

In [ ]:
seg = seg >0
out=seg.astype(np.uint8)
out[out>0]=255
writer = OmeTiffWriter('../../data/LAMP1/result/test_segmentation.tiff')
writer.save(out)